## LSTM Part-of-Speech Tagger
### Step 1: Importing the <code>torch</code>  and other libraries

In [17]:
#torch stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#other stuff
import numpy as np
import glob, os, random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
torch.manual_seed(1)

### Step 2: Get the helper functions

In [30]:
# Read the data file
def read_data(filepath):
    data = []
    with open(filepath, 'r', encoding="utf8") as f:
        for line in f:
            data.append(tuple(zip(*[wt.split('/') for wt in line.strip().split()])))
    return data

# Get vocabs and tagset
def get_vocab_tagset(data):
    token_size, vocabs, tagset = 0, set(), set()
    for words, tags in data:
        token_size += len(words)
        vocabs.update(words)
        tagset.update(tags)
    return vocabs, tagset, token_size

# convert the sequencies to indexes and tensors
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] if w in to_ix else 0 for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

# indexing list elements: use this for vocabs and tagset
def to_index(aList):
    elem_to_idx = {e:aList.index(e)+1 for e in aList}
    elem_to_idx['Unknown'] = 0  
    idx_to_elem = {i:e for e,i in elem_to_idx.items()}
    return elem_to_idx, idx_to_elem

# import numpy as np
def score_to_tag(tag_scores, i_to_tag):
    tagged=[]
    for preds in tag_scores:
        preds = list(np.array(preds))
        idx = preds.index(max(preds))
        tagged.append(f"{i_to_tag[idx]}")
    return tagged

# tagged_files = [#"igbo/ITC5.coarse*", "igbo/fiction.coarse*", #coarse
#                 #"igbo/ITC5.fine*", "igbo/fiction.fine*", #fine
#                 "welsh/*pos_coarse*",
#                 #"welsh/*sem_coarse*", 
#                 #"welsh/*both_fine*", "welsh/*both_coarse*",
#                 #"english/corpus.small"
#                 #"swedish/*coarse*"
#                 ]
# tagged data file
tagged_data_file = "all_corpora/welsh/cy_pos_coarse_tagged"

### Step 3: Preparing the data

In [23]:
# Read training data for each folder
print(f"\nReading {tagged_data_file} ...", end='')
data = read_data(tagged_data_file)
vocabs, tagset, token_size = get_vocab_tagset(data)

# #Uncomment below for data statistics
# print(f"\nSentences:\t{len(training_data)}\nToken size:\t{token_size}\nVocab size:\t{len(vocabs)}\nTagset size:\t{len(tagset)}")

# Shuffle and split data
random.seed(7)
random.shuffle(data)
test_size = 0.1 #90% of data for training; 10% of data for testing
train_set, test_set = data[:-int(len(data)*test_size)], data[-int(len(data)*test_size):]

EMBEDDING_DIM = 100
HIDDEN_DIM = 5    
word_to_idx, idx_to_word = to_index(list(vocabs))
tag_to_idx, idx_to_tag = to_index(list(tagset))


Reading all_corpora/welsh/cy_pos_coarse_tagged ...

### Step 4: Creating the model

In [24]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

### Step 5: Training the model

In [32]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_idx), len(tag_to_idx))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()

for epoch in range(10):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in train_set:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_idx)
        targets = prepare_sequence(tags, tag_to_idx)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        y_gold, y_pred = [], []
        for sent in test_set:
            y_gold.extend(sent[1])
            inputs = prepare_sequence(sent[0], word_to_idx)
            tag_scores = model(inputs)
            y_pred.extend(score_to_tag(tag_scores, idx_to_tag))
        score = [yg==yp for yg,yp in zip(y_gold, y_pred)]
        print(f"Epoch:{epoch:02d} = {sum(score)/len(score)*100:.2f}%|{accuracy_score(y_gold, y_pred)*100:.2f}%")

Epoch:0 = 37.17%
Epoch:1 = 43.85%
Epoch:2 = 51.80%
Epoch:3 = 55.95%
Epoch:4 = 58.09%
Epoch:5 = 60.43%
Epoch:6 = 61.30%
Epoch:7 = 61.90%
Epoch:8 = 62.77%
Epoch:9 = 63.77%
Epoch:10 = 63.97%
Epoch:11 = 64.10%
Epoch:12 = 65.24%
Epoch:13 = 66.31%
Epoch:14 = 67.05%
Epoch:15 = 67.91%
Epoch:16 = 68.38%
Epoch:17 = 69.18%
Epoch:18 = 69.45%
Epoch:19 = 69.79%
Epoch:20 = 70.39%
Epoch:21 = 70.12%
Epoch:22 = 70.05%
Epoch:23 = 70.39%
Epoch:24 = 69.99%
Epoch:25 = 70.45%
Epoch:26 = 70.79%
Epoch:27 = 70.92%
Epoch:28 = 71.26%
Epoch:29 = 71.32%
Epoch:30 = 71.39%
Epoch:31 = 71.26%
Epoch:32 = 71.32%
Epoch:33 = 71.26%
Epoch:34 = 71.26%
Epoch:35 = 71.52%
Epoch:36 = 71.79%
Epoch:37 = 71.86%
Epoch:38 = 72.39%
Epoch:39 = 72.13%
Epoch:40 = 72.19%
Epoch:41 = 72.33%
Epoch:42 = 72.66%
Epoch:43 = 72.33%
Epoch:44 = 72.66%
Epoch:45 = 72.53%
Epoch:46 = 72.53%
Epoch:47 = 72.39%
Epoch:48 = 72.46%
Epoch:49 = 72.59%
Epoch:50 = 72.66%
Epoch:51 = 72.59%
Epoch:52 = 72.86%
Epoch:53 = 72.73%
Epoch:54 = 73.33%
Epoch:55 = 73.13%
Ep

# ***** Forget about anything below here *****

In [25]:
# print("\nScores after training:")
def test_score():
    with torch.no_grad():
        inputs = prepare_sequence(untagged_sent, word_to_idx)
        tag_scores = model(inputs)
        model_tagged = score_to_tag(untagged_sent, tag_scores, idx_to_tag)
        #print(" ".join(model_tagged))
        scores = [t1==t2 for t1,t2 in zip(tagged_sent, model_tagged)]
        return(f"Score:\t{sum(scores)/len(scores)*100:.2f}%")
test_score()

NameError: name 'untagged_sent' is not defined

In [9]:
with open('all_corpora/welsh/cy_pos_fine_tagged', 'w', encoding='utf8') as f1:
    with open('all_corpora/welsh/cy_both_fine_tagged', 'r', encoding='utf8') as f2:
        for line in f2:
            f1.write(" ".join("/".join(wt.split('|',1)) for wt in line.split())+"\n")

In [15]:
import re
with open('all_corpora/welsh/cy_sem_coarse_tagged', 'w', encoding='utf8') as f1:
    with open('all_corpora/welsh/cy_both_fine_tagged', 'r', encoding='utf8') as f2:
        for line in f2:
            words, tags = tuple(zip(*[wt.split('/',1) for wt in line.strip().split()]))
#             pos_tags = [tag.split('|')[0] for tag in tags]
            sem_tags = [re.findall(r'[A-Za-z]+\d*',tag.split('|',1)[1])[0] for tag in tags]
#             coarse_pos_sem = ["|".join(ps) for ps in zip(pos_tags,sem_tags)]
#             word_coarse_pos_sem = [f"{w}/{t}" for w,t in zip(words, coarse_pos_sem)]
            word_coarse_sem = [f"{w}/{t}" for w,t in zip(words, sem_tags)]
#             print(f"{words}\n\n{tags}\n\n{both}\n\n{' '.join(word_both_tags)}")
#             print(f"{i}\t{' '.join(word_coarse_pos_sem)}")
            f1.write(f"{' '.join(word_coarse_sem)}\n")
#             break